# Table of Contents
 <p><div class="lev1"><a href="#Import-Python-modules"><span class="toc-item-num">1 - </span>Import <code>Python</code> modules</a></div><div class="lev1"><a href="#Set-CONSTANTS"><span class="toc-item-num">2 - </span>Set <em>CONSTANTS</em></a></div><div class="lev1"><a href="#List-Exogenous-SymPy-Variables:"><span class="toc-item-num">3 - </span>List Exogenous <code>SymPy</code> Variables:</a></div><div class="lev1"><a href="#Build-Struction-of-Model-for-Valuation-(conditional-upon-survival)"><span class="toc-item-num">4 - </span>Build Struction of Model for Valuation <em>(conditional upon survival)</em></a></div><div class="lev1"><a href="#Calculate-Actual-Values"><span class="toc-item-num">5 - </span>Calculate Actual Values</a></div>

# Import `Python` modules

In [1]:
from __future__ import print_function
import numpy
import pandas
from pprint import pprint
import sympy

from FinSymPy.Valuation import \
    terminal_value as tv, \
    present_value as pv, \
    net_present_value as npv
    
from HelpyFuncs.SymPy import sympy_eval_by_theano

Using gpu device 0: GeForce GT 750M


# Set _CONSTANTS_

In [2]:
# Revenue Y-o-Y Growth Rates
REV_GROWTH_RATES = [numpy.nan, numpy.nan, .5, .4, .3, .2, .2, .2, .2, .2, .2, .2, .05]

# EBIAT / Revenue Ratios
EBIAT_OVER_REVENUE_RATIOS = [numpy.nan, 0., .04, .05, .06, .06, .06, .06, .06, .06, .06, .06, .06]

# Further Investments & Working Capital Increases / Revenue Ratio
FIXED_ASSETS_AND_WORKING_CAP_OVER_REVENUE_RATIO = -.5

# DISCOUNT RATE
DISCOUNT_RATE = .174

# Long-Term Growth Rate
LONG_TERM_GROWTH_RATE = .04

# Year 0
YEAR_0 = 2003

# Number of Pro-Forma Years EXCLUDING Year 0
NB_PRO_FORMA_YEARS = 12
FINAL_PRO_FORMA_YEAR = YEAR_0 + NB_PRO_FORMA_YEARS

# List Exogenous `SymPy` Variables:

In [3]:
# Revenue
Rev = sympy.symbols('Rev_%d:%d' % (YEAR_0, YEAR_0 + 2))
print('Given Revenue variables:\n', Rev)

Rev_YoY_growth_rates = \
    (sympy.nan, sympy.nan) + \
    sympy.symbols('Rev_g_%d:%d' % (YEAR_0 + 2, FINAL_PRO_FORMA_YEAR + 1))
print('\nAssumed Revenue Y-o-Y Growth Rate variables:\n', Rev_YoY_growth_rates)


# EBIAT
EBIAT_over_Rev_ratios = \
    (sympy.nan,) + \
    sympy.symbols('EBIAT_over_Rev_%d:%d' % (YEAR_0 + 1, FINAL_PRO_FORMA_YEAR + 1))
print('\nAssumed EBIAT / Rev ratio variables:\n', EBIAT_over_Rev_ratios)


# Fixed Assets & Working Capital
FAnWC_over_Rev_ratio = sympy.Symbol('FAnWC_over_Rev')
print('\n(Fixed Assets + Working Capital) / Revenue ratio variable:\n', FAnWC_over_Rev_ratio)


# Discount Rate
discount_rate = sympy.Symbol('discount_rate')
print('\nDiscount Rate variable:\n', discount_rate)


# Long-Term Growth Rate
long_term_growth_rate = sympy.Symbol('long_term_growth_rate')
print('\nLong-Term Growth Rate variable:\n', long_term_growth_rate)


# assemble all exogenous variables into a collection
exo_vars = \
    Rev + Rev_YoY_growth_rates[2:] + \
    EBIAT_over_Rev_ratios[1:] + \
    (FAnWC_over_Rev_ratio,
     discount_rate,
     long_term_growth_rate)

Given Revenue variables:
 (Rev_2003, Rev_2004)

Assumed Revenue Y-o-Y Growth Rate variables:
 (nan, nan, Rev_g_2005, Rev_g_2006, Rev_g_2007, Rev_g_2008, Rev_g_2009, Rev_g_2010, Rev_g_2011, Rev_g_2012, Rev_g_2013, Rev_g_2014, Rev_g_2015)

Assumed EBIAT / Rev ratio variables:
 (nan, EBIAT_over_Rev_2004, EBIAT_over_Rev_2005, EBIAT_over_Rev_2006, EBIAT_over_Rev_2007, EBIAT_over_Rev_2008, EBIAT_over_Rev_2009, EBIAT_over_Rev_2010, EBIAT_over_Rev_2011, EBIAT_over_Rev_2012, EBIAT_over_Rev_2013, EBIAT_over_Rev_2014, EBIAT_over_Rev_2015)

(Fixed Assets + Working Capital) / Revenue ratio variable:
 FAnWC_over_Rev

Discount Rate variable:
 discount_rate

Long-Term Growth Rate variable:
 long_term_growth_rate


# Build Struction of Model for Valuation _(conditional upon survival)_

Revenue:

In [4]:
for g in Rev_YoY_growth_rates[2:]:
    Rev += ((1 + g) * Rev[-1],)

EBIAT:

In [5]:
EBIAT = map(
    lambda x: reduce(lambda u, v: u * v, x),
    zip(EBIAT_over_Rev_ratios, Rev))

Fixed Assets & Working Capital balance and its changes:

In [6]:
FixedAssets_and_WorkingCapital = \
    [FAnWC_over_Rev_ratio * rev for rev in Rev]
    
FixedAssets_and_WorkingCapital_changes = \
    [sympy.nan] + \
    [FixedAssets_and_WorkingCapital[i + 1] - FixedAssets_and_WorkingCapital[i]
     for i in range(NB_PRO_FORMA_YEARS)]

Free Cash Flows before Terminal Value:

In [7]:
FCF_before_TV = \
    map(lambda x: x[0] - x[1],
        zip(EBIAT, FixedAssets_and_WorkingCapital_changes))

Terminal Value:

In [8]:
TV = tv(FCF_before_TV[-1], discount_rate, long_term_growth_rate)

Valuation:

In [9]:
val_of_FCF_before_TV = npv([0] + FCF_before_TV[1:], discount_rate)

val_of_TV = pv(TV, discount_rate, NB_PRO_FORMA_YEARS)

val = val_of_FCF_before_TV + val_of_TV

# Calculate Actual Values

Import data:

In [10]:
data_df = \
    pandas.read_csv(
        'data/RightNow Technologies.csv',
        index_col='Year')
    
data_df

,Revenue
Year,
2003,29300000
2004,60000000


Exogenous Variables' values:

In [11]:
exo_vars_values_dict = \
    dict(
        FAnWC_over_Rev=FIXED_ASSETS_AND_WORKING_CAP_OVER_REVENUE_RATIO,
        discount_rate=DISCOUNT_RATE,
        long_term_growth_rate=LONG_TERM_GROWTH_RATE)
    
exo_vars_values_dict.update(
    dict(
        zip([i.name for i in Rev[:2]],
            (data_df.Revenue.astype(numpy.float32) / 1e6).tolist()) +
        zip([i.name for i in Rev_YoY_growth_rates[2:]],
            REV_GROWTH_RATES[2:]) +
        zip([i.name for i in EBIAT_over_Rev_ratios[1:]],
            EBIAT_OVER_REVENUE_RATIOS[1:])))

pprint(exo_vars_values_dict)

{'EBIAT_over_Rev_2004': 0.0,
 'EBIAT_over_Rev_2005': 0.04,
 'EBIAT_over_Rev_2006': 0.05,
 'EBIAT_over_Rev_2007': 0.06,
 'EBIAT_over_Rev_2008': 0.06,
 'EBIAT_over_Rev_2009': 0.06,
 'EBIAT_over_Rev_2010': 0.06,
 'EBIAT_over_Rev_2011': 0.06,
 'EBIAT_over_Rev_2012': 0.06,
 'EBIAT_over_Rev_2013': 0.06,
 'EBIAT_over_Rev_2014': 0.06,
 'EBIAT_over_Rev_2015': 0.06,
 'FAnWC_over_Rev': -0.5,
 'Rev_2003': 29.299999,
 'Rev_2004': 60.0,
 'Rev_g_2005': 0.5,
 'Rev_g_2006': 0.4,
 'Rev_g_2007': 0.3,
 'Rev_g_2008': 0.2,
 'Rev_g_2009': 0.2,
 'Rev_g_2010': 0.2,
 'Rev_g_2011': 0.2,
 'Rev_g_2012': 0.2,
 'Rev_g_2013': 0.2,
 'Rev_g_2014': 0.2,
 'Rev_g_2015': 0.05,
 'discount_rate': 0.174,
 'long_term_growth_rate': 0.04}


Function to calculate values from `SymPy` expression:

In [12]:
def calc(x):
    if isinstance(x, (list, tuple)):
        return [float(calc(i)) for i in x]
    else:
        return float(sympy_eval_by_theano(x, symbols=exo_vars, **exo_vars_values_dict))

Valuation Calculations:

In [13]:
print('VALUATION CONDITIONAL UPON SURVIVAL')
print('___________________________________\n')

print('(Fixed Assets + Working Capital) / Revenue =   %s\n' % '{:.0%}'.format(calc(FAnWC_over_Rev_ratio)))

print('Discount Rate =   %s\n' % '{:.1%}'.format(calc(discount_rate)))

print('Long-Term Growth Rate =   %s\n' % '{:.0%}'.format(calc(long_term_growth_rate)))

print('Valuation of FCF before TV =   $%.2f MM' % calc(val_of_FCF_before_TV))
print('Valuation of TV =   $%.2f MM' % calc(val_of_TV))
print('\nVALUATION | survival =   $%.2f MM\n' % calc(val))

print('Detailed Cash Flow Calcs:')
val_calcs_df = pandas.DataFrame(index=['Year 0'] + range(YEAR_0 + 1, FINAL_PRO_FORMA_YEAR + 1))
pandas.options.display.float_format = '{:,.2f}'.format

val_calcs_df['Revenue Y-o-Y Growth Rate'] = [numpy.nan, numpy.nan] + calc(Rev_YoY_growth_rates[2:])
val_calcs_df['Revenue'] = calc(Rev)
val_calcs_df['EBIAT / Revenue'] = [numpy.nan] + calc(EBIAT_over_Rev_ratios[1:])
val_calcs_df['EBIAT'] = [numpy.nan] + calc(EBIAT[1:])
val_calcs_df['Fixed Assets & Wkg. Cap.'] = calc(FixedAssets_and_WorkingCapital)
val_calcs_df['Fixed Assets & Wkg. Cap. chg.'] = [numpy.nan] + calc(FixedAssets_and_WorkingCapital_changes[1:])
val_calcs_df['FCF before Terminal Value'] = [numpy.nan] + calc(FCF_before_TV[1:])
val_calcs_df['Terminal Value'] = numpy.nan
val_calcs_df.loc[FINAL_PRO_FORMA_YEAR, 'Terminal Value'] = calc(TV)

val_calcs_df.T

VALUATION CONDITIONAL UPON SURVIVAL
___________________________________

(Fixed Assets + Working Capital) / Revenue =   -50%

Discount Rate =   17.4%

Long-Term Growth Rate =   4%

Valuation of FCF before TV =   $158.77 MM
Valuation of TV =   $58.48 MM

VALUATION | survival =   $217.25 MM

Detailed Cash Flow Calcs:


,Year 0,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenue Y-o-Y Growth Rate,nan,nan,0.50,0.40,0.30,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.05
Revenue,29.30,60.00,90.00,126.00,163.80,196.56,235.87,283.05,339.66,407.59,489.10,586.93,616.27
EBIAT / Revenue,nan,0.00,0.04,0.05,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
EBIAT,nan,0.00,3.60,6.30,9.83,11.79,14.15,16.98,20.38,24.46,29.35,35.22,36.98
Fixed Assets & Wkg. Cap.,-14.65,-30.00,-45.00,-63.00,-81.90,-98.28,-117.94,-141.52,-169.83,-203.79,-244.55,-293.46,-308.14
Fixed Assets & Wkg. Cap. chg.,nan,-15.35,-15.00,-18.00,-18.90,-16.38,-19.66,-23.59,-28.30,-33.97,-40.76,-48.91,-14.67
FCF before Terminal Value,nan,15.35,18.60,24.30,28.73,28.17,33.81,40.57,48.68,58.42,70.10,84.13,51.65
Terminal Value,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,400.86
